# **에이전트(Agent) 구축하기**

언어 모델(LLM) 자체는 **행동을 수행할 수 없습니다**  그저 텍스트를 출력할 뿐입니다.  
LangChain의 주요 사용 사례 중 하나는 **에이전트(Agents)**를 만드는 것입니다.  

**에이전트(Agents)**는 **LLM**을 **추론 엔진**으로 사용하여 어떤 행동을 취해야 하는지, 그리고 행동을 수행하는 데 필요한 입력이 무엇인지 결정하는 시스템입니다.  
행동을 실행한 후 결과를 LLM에 다시 전달하여 **더 많은 행동이 필요한지** 또는 **작업을 종료해도 되는지**를 판단할 수 있습니다. 이는 종종 **도구 호출(tool-calling)**을 통해 이루어집니다.  

---

이 노트북에서는 **검색 엔진과 상호작용할 수 있는 에이전트**를 구축합니다.  
- 사용자는 이 에이전트에게 질문을 할 수 있습니다.  
- 에이전트가 검색 도구를 호출하는 과정을 볼 수 있습니다.  
- 에이전트와 여러 차례 대화를 나눌 수 있습니다.  

---

## **엔드 투 엔드 에이전트 (End-to-end Agent)**

아래 코드는 **도구(tool)**를 사용하여 어떤 행동을 취해야 할지 결정하는 LLM을 활용한 **완전히 기능하는 에이전트**를 보여줍니다.  
- 이 에이전트는 **일반적인 검색 도구**를 사용할 수 있습니다.  
- **대화 메모리(conversational memory)**를 갖추고 있어 다중 턴 대화가 가능합니다.
- 도구 정의 - 우리의 주요 선택 도구는 **Tavily**입니다. Tavily는 **검색 엔진**으로, LangChain에는 Tavily 검색 엔진을 쉽게 사용할 수 있도록 지원하는 **내장 도구**가 포함되어 있습니다.


In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv \
# langgraph

In [ ]:
# !pip install -qU tavily-python langgraph-checkpoint-sqlite

In [ ]:
# LangChain 추적(Tracing) 설정 활성화

In [ ]:
# from langchain_anthropic import ChatAnthropic

##  **언어 모델과 도구 정의**

먼저 언어 모델과 사용하려는 도구를 생성해야 합니다. 우리의 주요 선택 도구는 **Tavily**입니다. Tavily는 **검색 엔진**으로, LangChain에는 Tavily 검색 엔진을 쉽게 사용할 수 있도록 지원하는 **내장 도구**가 포함되어 있습니다.

In [ ]:
# model = ChatAnthropic(model="claude-3-5-sonnet-20241022")
# 검색 도구 설정 (최대 검색 결과 2개)
# 사용할 모든 도구를 하나의 리스트에 넣어서 나중에 참조할 수 있도록 설정

### 1) 도구 없이 LLM 만 호출할 경우

### 2) Agent 구축 않고 LLM에 도구 목록만 전달할 경우
이제 이 모델이 도구 호출을 수행하도록 활성화하기 위해 `.bind_tools`를 사용하여 언어 모델에 이러한 도구에 대한 지식을 제공합니다.

In [ ]:
# `bind_tools` 메서드를 사용하여 언어 모델이 특정 도구 목록을 사용할 수 있도록 설정합니다.

모델은 우리가 **Tavily Search** 도구를 호출하라고 알려주며 도구 호출에 필요한 argument를 만들어 줍니다.  
하지만 이것은 아직 해당 도구를 호출한 것이 아니라, 단지 호출해야 한다고 알려주는 것입니다. 실제로 도구를 호출하려면 **에이전트(agent)**를 만들어야 합니다.

## Agent 생성

- LLM은 **상태를 저장하지 않는(stateless)** 이므로 이전 상호작용을 기억하지 않습니다. 에이전트에 메모리를 추가하려면 **체크포인터(checkpointer)**를 전달해야 합니다. 또한 에이전트를 호출할 때 `thread_id`를 함께 전달해야 합니다. 이를 통해 에이전트는 어느 스레드(대화)에서부터 상호작용을 이어가야 하는지 알 수 있습니다.

- ReAct 에이전트를 구성하기 위해 Langgraph에서 제공하는 고수준 인터페이스(high-level interface)인 `create_react_agent`를 사용합니다. 이를 통해 에이전트 로직을 원하는 대로 세부적으로 수정할 수 있습니다. `create_react_agent`는 내부적으로 `.bind_tools`를 호출해 도구를 모델에 연결합니다.

In [ ]:
# 에이전트의 메모리 저장 객체 생성
# 에이전트 실행 객체 생성
# 에이전트 실행 구성 설정

## 스트리밍 메시지  

우리는 `.invoke`를 사용해 에이전트를 호출하고 최종 응답을 받는 방법을 알고 있습니다. 하지만 에이전트가 여러 단계를 실행해야 할 경우 시간이 오래 걸릴 수 있습니다.  
중간 진행 상황을 보여주기 위해, 발생하는 메시지를 스트리밍 형태로 실시간 전송할 수 있습니다.

In [ ]:
# 에이전트 실행

#### 메모리가 필요한 질문 

#### ReAct 패턴의 질문/응답 
질문이 단순 정보 조회가 아닌, 추론과 행동을 반복해야 해결 가능한 질문
예) 내가 사는 곳의 날씨를 알려줘 
- Reasoning(추론) - 사용자의 위치를 알아야 하고, 날씨 API에서 날씨 정보를 가져와야 한다. (LLM 역할)  
- Acting(행동) - TavilySearchResults API 호출 (LangChin 역할)  
- Observation(관찰) - API로부터 "오늘 서울은 맑음"이라는 정보를 받았다. 이 것을 LLM에 다시 전달. (LangChain 역할)
- Reasoning(추론) - "이제 사용자에게 서울의 날씨가 맑다고 답변해야 한다."  (LLM의 역할)  
- Answer(응답) - "서울의 날씨는 맑음입니다." (LLM의 역할)  

Example [LangSmith trace](https://smith.langchain.com/o/351c6cd9-1396-5c74-9478-1ee6a22a6433/projects/p/acec9d4d-4978-4597-adff-789cd42e200f?timeModel=%7B%22duration%22%3A%227d%22%7D&peek=17495aca-857d-4d57-95e7-6c05ac8348d7)

새로운 대화를 시작하려면 사용 중인 `thread_id`를 변경하기만 하면 됩니다.